In [2]:
# Подключаем google disk, копируем файлы в colab
from google.colab import drive
drive.mount('/content/drive')
!cp -r /content/drive/MyDrive/app/src/ /content/src
!cp -r /content/drive/MyDrive/app/raw_data/ /content/raw_data
!pip install implicit

Mounted at /content/drive
     |████████████████████████████████| 1.1MB 8.1MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp36-cp36m-linux_x86_64.whl size=3419398 sha256=a0f5825dee647924ba9b8b7b40b96852a4f3f9374acb8b7d6b1e673de18c0c92
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [22]:
# Загрузка данных
data = pd.read_csv('./raw_data/retail_train.csv') # Данные о покупках пользователей
item_features = pd.read_csv('./raw_data/product.csv') # Данные о товарах
user_features = pd.read_csv('./raw_data/hh_demographic.csv') # Данные о пользователях
data_test = pd.read_csv('./raw_data/retail_test1.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

In [23]:
# len_week = len(pd.unique(data['week_no']))
# len_first_lvl = len_week // 2
# len_secon_lvl = (len_week - len_first_lvl) // 2
# len_three_lvl = len_week - (len_first_lvl + len_secon_lvl)
# print(len_first_lvl + len_secon_lvl + len_three_lvl)

95


In [24]:
# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

In [47]:
# Функция для выдачи рекомендаций различными методами 
def recomend_f(i_user_features, recomen, data_val2, N=5):
  res_als = 0
  res_own = 0
  res_similar_items = 0
  res_similar_users = 0

  recomend = []
  # print(0)
  for user in data_val2.groupby('user_id')['item_id'].count().index:#i_user_features['user_id']:
      # print(1)
      bought_list = data_val2.loc[data_val2['user_id'] == user].groupby(['item_id'])['quantity'].count().index
      # print('als')
      rec_als = recomen.get_als_recommendations(user, N=N)
      res_als += precision_at_k(rec_als, bought_list, N)
      # print('own')
      rec_own = recomen.get_own_recommendations(user, N=N)
      res_own += precision_at_k(rec_own, bought_list, N)
      # print('sim_item')
      rec_similar_items = recomen.get_similar_items_recommendation(user, N=N)
      res_similar_items += precision_at_k(rec_similar_items, bought_list, N)

      try:
        rec_similar_users = recomen.get_similar_users_recommendation(user, N=N)
        res_similar_users += precision_at_k(rec_similar_users, bought_list, N)
      except Exception as e:
        res_similar_users += 0
      recomend.append([user, rec_own])
      # user_features.loc[user_features['user_id']== user]['recomend'] = rec_own
        # print(user)
        # print(e)
  print(f'Precision ALS = {res_als/len(i_user_features)}')
  print(f'Precision own = {res_own/len(i_user_features)}')
  print(f'Precision similar items = {res_similar_items/len(i_user_features)}')
  print(f'Precision similar users = {res_similar_users/len(i_user_features)}')
  print('\n')
  return recomend

In [48]:
# Подбор наилучших параметров
count_item = [10000, 7500, 5000, 2500, 1000, 500]
N_list = [5, 10, 20, 30]
for n_tmp in N_list:
  print(f'Count recomendation: {n_tmp}')
  for i in count_item:
    print(f'Count prefilter items: {i}:')
    data_train_tmp = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=i)
    # data_train_tmp_lvl2 = prefilter_items(data_train_lvl_2, item_features=item_features, take_n_popular=i)
    recommender = MainRecommender(data_train_tmp)
    recomend_f(user_features, recommender, data_train_lvl_2, N=n_tmp)

Count recomendation: 5
Count prefilter items: 10000:



Precision ALS = 0.31735330836454123
Precision own = 0.8144818976279632
Precision similar items = 0.19775280898876388
Precision similar users = 0.03220973782771529


Count prefilter items: 7500:



Precision ALS = 0.3153558052434428
Precision own = 0.8144818976279632
Precision similar items = 0.19525593008739062
Precision similar users = 0.031710362047440634


Count prefilter items: 5000:



Precision ALS = 0.31910112359550247
Precision own = 0.8144818976279632
Precision similar items = 0.19126092384519372
Precision similar users = 0.03220973782771528


Count prefilter items: 2500:



Precision ALS = 0.30611735330836204
Precision own = 0.8144818976279632
Precision similar items = 0.20174781523096108
Precision similar users = 0.034706616729088556


Count prefilter items: 1000:


...


Precision ALS = 0.2412401165210119
Precision own = 0.46525176862255735
Precision similar items = 0.14461090303786833
Precision similar users = 0.031294215563878784


Count prefilter items: 5000:



Precision ALS = 0.24015813566374994
Precision own = 0.46525176862255735
Precision similar items = 0.14806491885143475
Precision similar users = 0.031169371618810126


Count prefilter items: 2500:



Precision ALS = 0.24277985851019193
Precision own = 0.46525176862255735
Precision similar items = 0.1517270079067824
Precision similar users = 0.030461922596754322


Count prefilter items: 1000:



Precision ALS = 0.24365376612567277
Precision own = 0.46529338327091363
Precision similar items = 0.14606741573033613
Precision similar users = 0.029671244277986106


Count prefilter items: 500:



Precision ALS = 0.2601331668747358
Precision own = 0.43890969621306847
Precision similar items = 0.13736995422388595
Precision similar users = 0.05401581356637561




In [49]:
# Наилучшие рекомендации получились при отборе 500 популярных товаров
data_train_lvl_1 = prefilter_items(data, item_features=item_features, take_n_popular=500)
# Делаем рекомендации на основе всех данных и валедируем на тестовом DataSet
recommender = MainRecommender(data_train_lvl_1)
recomend = recomend_f(user_features, recommender, data_test, N=5)


Precision ALS = 0.24619225967540453
Precision own = 0.6719101123595462
Precision similar items = 0.14531835205992608
Precision similar users = 0.057428214731585624




На тестовом DataSet precision@5 = 0,67, для рекомендации на основе собственных покупок пользователя.

In [61]:
pd.DataFrame(recomend, columns=['user_id', 'rec_i']).to_csv('recomend.csv', index=False)

In [63]:
!cp ./recomend.csv /content/drive/MyDrive/app/recomend.csv